In [1]:
import pandas as pd
import numpy as np
import scipy.stats as st
finetune = pd.read_csv('data/sgsm_train.csv')
finetune

,Unnamed: 0,question,solution,answer,topic
0,0,"LeBron James is 6'9"" tall. 1 inch is 2.54 cm. ...","def solution():\n #LeBron James is 6'9"" tal...",205.74,basketball
1,1,Captain America has 1200 ryo. He spends 400 ry...,def solution():\n #Captain America started ...,300.0,Captain America
2,3,3 scoops of ice cream cost $4. \nHow much does...,def solution():\n #3 scoops of ice cream co...,13.333333333333332,ice cream
3,5,A volleyball team is 6 players. \n3 of the pla...,def solution():\n #3 of the players are 16 ...,16.666666666666668,volleyball
4,6,"Superman can fly at a speed of 1,000 miles per...",def solution():\n #Superman can fly at a sp...,0.2,Superman
...,...,...,...,...,...
2088,1901,30 unicorns are in a herd. 10 unicorns in the ...,def solution():\n #Total number of unicorns...,17,unicorns
2089,1902,A pizza has 8 slices. 4 of the slices have pep...,def solution():\n #A pizza has 8 slices\n ...,4,pizza
2090,1903,12000 people were at a rally. 10000 of them we...,def solution():\n #Total number of people a...,500,unspecified
2091,1904,A bakery made 300 muffins. 150 of the muffins ...,def solution():\n #Total muffins\n total...,60,unspecified


## Llama 3 Zero Shot

In [15]:
file_path = 'llama3_8b_questions.txt'
data = []

with open(file_path, 'r') as file:
    text = file.read()
    sections = text.split("Topic: ")
    #sections = text.split("### Response:")
    del sections[0]  # Remove the initial part before the question
    total_questions = len(sections)
    non_zero = total_questions
    for section in sections:
        if "Question" not in section:
            non_zero-=1
        try:
            topic = section.split("### Response:")[0].strip()
            section = section.split("### Response:")[1].strip()
            question = section.split("Question:")[1].strip()
            question = question.split("Solution:")[0].strip()
            solution = section.split("Solution:")[1].strip()
            if 'while' in solution:
                pass
            if "##" in question:
                question = question.split("##")[0]
#             if "\n\n" in question:
#                 question = question.split("\n\n")
#                 question = "\n".join(question)
            if "\nBel" in solution:
                solution = solution.split("\nBel")[0]
            if "##" in solution:
                solution = solution.split("##")[0]
            if "return" not in solution:
                try: 
                    solution = solution + '\n' + '    return result'
                    solution_text = solution
                    try:
                            # Execute the Python code
                        exec(solution)
                        answer = solution()
                        if answer==None:
                            continue
                        data.append({"question": question, "solution": solution_text, 'answer': answer, 'topic': topic})
                    except: 
                        try: #pick up questions that did not indent the code block
                            solution_new = ""
                            lines = solution.split('\n')
                            indent = False
                            for line in lines:
                                if line.startswith('def solution():'):
                                    solution_new += line + '\n'

                                if not line.startswith('def solution():') and not line.startswith('    return result'):
                                    solution_new += '    ' + line + '\n'

                                if line.startswith('    return result'):
                                    solution_new += line
                            solution_text = solution_new
                            # Execute the Python code
                            exec(solution_new)
                            answer = solution()
                            if answer==None:
                                continue
                            data.append({"question": question, "solution": solution_text, 'answer': answer, "topic": topic})
                        except:
                                pass
                except:
                    pass
            if "return" in solution:
                try: 
                    solution_text = solution
                    try:
                        # Execute the Python code
                        exec(solution)
                        answer = solution()
                        if answer==None:
                            continue
                        data.append({"question": question, "solution": solution_text, 'answer': answer, "topic": topic})
                    except: 
                        try: #pick up questions that did not indent the code block
                            solution_new = ""
                            lines = solution.split('\n')
                            indent = False
                            for line in lines:
                                if line.startswith('def solution():'):
                                    solution_new += line + '\n'

                                if not line.startswith('def solution():') and not line.startswith('    return result'):
                                    solution_new += '    ' + line + '\n'

                                if line.startswith('    return result'):
                                    solution_new += line
                            solution_text = solution_new
                            # Execute the Python code
                            exec(solution_new)
                            answer = solution()
                            if answer==None:
                                continue
                            data.append({"question": question, "solution": solution_text, 'answer': answer, "topic": topic})
                        except:
                            pass
                except:
                    pass
        except:
            pass
df1 = pd.DataFrame.from_dict(data)
result1 = "Percentage of Questions with Executable Code: " + str((len(df1)/total_questions)*100)
result2 = "Percentage of Questions with Executable Code (excluding non generations): " + str((len(df1)/non_zero)*100)
print(result1)
print(result2)
print(len(df1), total_questions)

150
21.0
129.0
26.0


How many Grand Slam titles did Serena Williams win in 2017? 2
How many Grand Slam titles did Serena Williams win in 2017? 2
Enter Serena's weight in pounds:  2
Enter the weight of a serving of tuna fish in grams:  2


91.30434782608695% of Serena's titles are singles
quarterback is not quarterback
rate =  5
26 Hello Kitties
sword
It will take Train 1 0 minute(s) to overtake Train 2.
28
1.5
It will take Steve 12.5 digs to make a whole of blocks
['Harry', 'Neville', 'Draco']
['Harry', 'Neville', 'Draco']
The player with the most cards is Neville

I am Wonder Woman
Wonder Woman is wearing a Wonder Woman outfit
They Wonder Woman is wearing boots
The Wonder Woman completed the Day
The Wonder Woman decided to to put on her Wonder Woman boots again
The Wonder Woman goes into the city to save the world
12th
12th
11th
11th
4
Steve has  4 irons left
There are 262 cats in total.


Please enter a number between 0-200: 22


0.0
4.0
80.0
Chapter 1: 150 chapter_points
Harry points have made: 150
Chapter 2: 150 chapter_points
Harry points have made: 300
Harry points have made: 300
Harry points have made: 300
Chapter 5: 150 chapter_points
Harry points have made: 450
Percentage of Questions with Executable Code: 29.080000000000002
Percentage of Questions with Executable Code (excluding non generations): 43.48086124401914
1454 5000


In [8]:
print(f"Percentage of questions that have 0 as answer: {(len(df1[df1['answer']==0])/len(df1))*100}")

Percentage of questions that have 0 as answer: 4.607977991746905


In [84]:
df1 = df1.drop_duplicates(subset='question')
df1 = df1.dropna(subset = 'topic')

In [85]:
df1

,question,solution,answer,topic
8,Captain America is preparing his shield for ba...,def solution():\n #Captain America throws h...,8.33,Captain America
9,Captain Marvel flew a distance of 50 meters an...,def solution():\n #Captain Marvel flew 50 m...,85.0,planes
10,Captain Marvel has 500 shields. Her partner ha...,def solution():\n #Captain Marvel has 500 s...,900,Captain Marvel
11,Harry Potter lives at 4 Privet Drive. There a...,def solution():\n houses_street = 11\n ...,-17,Harry Potter
12,Naruto and his companions will need 100 chakra...,def solution():\n #We know that Naruto and ...,10,Naruto
...,...,...,...,...
1449,Serena is a 4 time Olympian. She took a bronze...,def solution():\n #Serena has won medals in...,7,Serena Williams
1450,"Green Ranger, the superhero, and Kat, the assi...",def solution():\n #Green Ranger and Kat hav...,280.0,Power Rangers
1451,There are 10 Power Rangers. 2 of them are from...,def solution():\n #total: 10 Power Rangers\...,14,Captain America
1452,There are 8 players on a soccer team. 7 of the...,def solution():\n #There are 8 players on a...,0,soccer


In [86]:
question_numbers = []
for i in range(0, len(df1)):
    question_numbers.append(f"llama3_q{i}")
df1['question_number'] = question_numbers
df1_an = df1.sample(n=250, random_state = 42)

In [92]:
df1_an = df1_an[~df1_an['question'].isin(finetune['question'])]
df1 = df1[~df1['question'].isin(finetune['question'])]
df1 = df1[~df1['question'].isin(df1_an['question'])]

In [90]:
df1_an

,question,solution,answer,topic,question_number
176,"LeBron James, an excellent basketball player, ...",def solution():\n #the challenge will never e...,The challenge will never end if it is left up ...,LeBron James,llama3_q168
614,An airplane has 12 seats on each row. 72 seats...,def solution():\n #An airplane has 12 seats...,5,planes,llama3_q605
557,A basketball league has 12 teams in a division...,def solution():\n #Total number of teams\n ...,300,basketball,llama3_q548
73,Harry is about to battle Voldemort. If Voldemo...,def solution():\n #Voldemort has 4000 point...,2000.0,Harry Potter,llama3_q65
637,There are 23 passenger cars on a train. 14 car...,def solution():\n #The train has 23 passeng...,9,trains,llama3_q628
...,...,...,...,...,...
204,"At an NBA game, LeBron James made 5 three-poin...",def solution():\n #LeBron James made 5 thre...,34,LeBron James,llama3_q196
1335,Superman ate 10% of the hamburgers at the burg...,def solution():\n #There are 100 hamburgers...,10.0,Superman,llama3_q1321
136,Darryl is running a candy machine at the airpo...,def solution():\n #Darryl starts with 200 g...,28,candy,llama3_q128
1220,You bring 30 cookies. 15 more cookies are brou...,def solution():\n #30 cookies brought by yo...,20,cookies,llama3_q1208


## MATHWELL8b Full Finetune

In [46]:
file_path = 'mathwell8b_full_questions.txt'
data = []

with open(file_path, 'r') as file:
    text = file.read()
    sections = text.split("Topic: ")
    #sections = text.split("### Response:")
    del sections[0]  # Remove the initial part before the question
    total_questions = len(sections)
    non_zero = total_questions
    for section in sections:
        if "Question" not in section:
            non_zero-=1
        try:
            topic = section.split("### Response:")[0].strip()
            section = section.split("### Response:")[1].strip()
            question = section.split("Question:")[1].strip()
            question = question.split("Solution:")[0].strip()
            solution = section.split("Solution:")[1].strip()
            if 'while' in solution:
                pass
            if "##" in question:
                question = question.split("##")[0]
#             if "\n\n" in question:
#                 question = question.split("\n\n")
#                 question = "\n".join(question)
            if "\nBel" in solution:
                solution = solution.split("\nBel")[0]
            if "##" in solution:
                solution = solution.split("##")[0]
            if "return" not in solution:
                try: 
                    solution = solution + '\n' + '    return result'
                    solution_text = solution
                    try:
                            # Execute the Python code
                        exec(solution)
                        answer = solution()
                        if answer==None:
                            continue
                        data.append({"question": question, "solution": solution_text, 'answer': answer, 'topic': topic})
                    except: 
                        try: #pick up questions that did not indent the code block
                            solution_new = ""
                            lines = solution.split('\n')
                            indent = False
                            for line in lines:
                                if line.startswith('def solution():'):
                                    solution_new += line + '\n'

                                if not line.startswith('def solution():') and not line.startswith('    return result'):
                                    solution_new += '    ' + line + '\n'

                                if line.startswith('    return result'):
                                    solution_new += line
                            solution_text = solution_new
                            # Execute the Python code
                            exec(solution_new)
                            answer = solution()
                            if answer==None:
                                continue
                            data.append({"question": question, "solution": solution_text, 'answer': answer, "topic": topic})
                        except:
                                pass
                except:
                    pass
            if "return" in solution:
                try: 
                    solution_text = solution
                    try:
                        # Execute the Python code
                        exec(solution)
                        answer = solution()
                        if answer==None:
                            continue
                        data.append({"question": question, "solution": solution_text, 'answer': answer, "topic": topic})
                    except: 
                        try: #pick up questions that did not indent the code block
                            solution_new = ""
                            lines = solution.split('\n')
                            indent = False
                            for line in lines:
                                if line.startswith('def solution():'):
                                    solution_new += line + '\n'

                                if not line.startswith('def solution():') and not line.startswith('    return result'):
                                    solution_new += '    ' + line + '\n'

                                if line.startswith('    return result'):
                                    solution_new += line
                            solution_text = solution_new
                            # Execute the Python code
                            exec(solution_new)
                            answer = solution()
                            if answer==None:
                                continue
                            data.append({"question": question, "solution": solution_text, 'answer': answer, "topic": topic})
                        except:
                            pass
                except:
                    pass
        except:
            pass
df2 = pd.DataFrame.from_dict(data)
result1 = "Percentage of Questions with Executable Code: " + str((len(df2)/total_questions)*100)
result2 = "Percentage of Questions with Executable Code (excluding non generations): " + str((len(df2)/non_zero)*100)
print(result1)
print(result2)
print(len(df2), total_questions)

(37, 39)
Percentage of Questions with Executable Code: 24.15
Percentage of Questions with Executable Code (excluding non generations): 88.78676470588235
483 2000


In [47]:
print(f"Percentage of questions that have 0 as answer: {(len(df2[df2['answer']==0])/len(df2))*100}")

Percentage of questions that have 0 as answer: 4.968944099378882


In [49]:
df2 = df2.drop_duplicates(subset='question')
df2 = df2[~df2['question'].isin(finetune['question'])]
question_numbers = []
for i in range(0, len(df2)):
    question_numbers.append(f"mathwell8b_full_q{i}")
df2['question_number'] = question_numbers
df2_an = df2.sample(n=min(250,len(df2)), random_state = 42)
df2_an

,question,solution,answer,topic,question_number
93,A pilot is flying a plane that weighs 1000 pou...,def solution():\n #The plane weighs 1000 po...,6000,planes,mathwell8b_full_q93
321,Jules owns 25 cars. 10 of them are sports cars...,def solution():\n #Total cars\n total_ca...,13,cars,mathwell8b_full_q320
55,20 unicorns were in a meadow. 5 of the unicorn...,def solution():\n #20 unicorns were in a me...,18,unicorns,mathwell8b_full_q55
30,A unicorn ate 30 dandelions and 12 blueberries...,def solution():\n #The unicorn ate 30 dande...,360,unicorns,mathwell8b_full_q30
453,1000000000 golden apples fell from the heavens...,def solution():\n #The total number of gold...,-1010000000,Wonder Woman,mathwell8b_full_q449
...,...,...,...,...,...
96,Black Panther has 52 vibranium bullets. He fir...,def solution():\n #Black Panther has 52 vib...,22,Black Panther,mathwell8b_full_q96
143,20 players on each team are on the field. 80% ...,def solution():\n #20 players on each team ...,3.2,lacrosse,mathwell8b_full_q143
239,200 scoops of ice cream are on the conveyor be...,def solution():\n #Total scoops of ice crea...,150,ice cream,mathwell8b_full_q239
437,10 friends are ordering pizza. Each of them wa...,def solution():\n #Number of friends\n f...,1.666667,pizza,mathwell8b_full_q434


In [51]:
df2

,question,solution,answer,topic,question_number
0,A player has 12000 experience points. Experien...,def solution():\n #The player has 12000 exp...,120.0,Minecraft,mathwell8b_full_q0
1,Hello Kitty has 500 pencils. 200 of the pencil...,def solution():\n #Hello Kitty has 500 penc...,150,Hello Kitty,mathwell8b_full_q1
2,Serena Williams has 200 trophies. 100 of them ...,def solution():\n #Total trophies\n tota...,100,Serena Williams,mathwell8b_full_q2
3,"10,000 dogs are barking. 1/4 of the dogs are R...",def solution():\n #Total dogs\n total_do...,500.0,dogs,mathwell8b_full_q3
4,300 Barbie dolls were in a toy store. 120 of t...,def solution():\n #300 Barbie dolls were in...,120,Barbie,mathwell8b_full_q4
...,...,...,...,...,...
478,Serena Williams has 200 Wimbledon titles. 100 ...,def solution():\n #Serena Williams has 200 ...,50,Serena Williams,mathwell8b_full_q474
479,"In volleyball, each team has 6 players on the ...",def solution():\n #Each team has 6 players\...,49.333333,volleyball,mathwell8b_full_q475
480,A 12-person soccer team has 32 shirts. \n8 of ...,def solution():\n #A 12-person soccer team ...,18,football,mathwell8b_full_q476
481,How many scoops of ice cream can you get from ...,def solution():\n #1 scoop of ice cream is ...,4.0,ice cream,mathwell8b_full_q477


## MATHWELL 8b Two Stage

In [93]:
file_path = 'mathwell8b_two_stage_questions.txt'
data = []

with open(file_path, 'r') as file:
    text = file.read()
    sections = text.split("Topic: ")
    #sections = text.split("### Response:")
    del sections[0]  # Remove the initial part before the question
    total_questions = len(sections)
    non_zero = total_questions
    for section in sections:
        if "Question" not in section:
            non_zero-=1
        try:
            topic = section.split("### Response:")[0].strip()
            section = section.split("### Response:")[1].strip()
            question = section.split("Question:")[1].strip()
            question = question.split("Solution:")[0].strip()
            solution = section.split("Solution:")[1].strip()
            if 'while' in solution:
                pass
            if "##" in question:
                question = question.split("##")[0]
#             if "\n\n" in question:
#                 question = question.split("\n\n")
#                 question = "\n".join(question)
            if "\nBel" in solution:
                solution = solution.split("\nBel")[0]
            if "##" in solution:
                solution = solution.split("##")[0]
            if "return" not in solution:
                try: 
                    solution = solution + '\n' + '    return result'
                    solution_text = solution
                    try:
                            # Execute the Python code
                        exec(solution)
                        answer = solution()
                        if answer==None:
                            continue
                        data.append({"question": question, "solution": solution_text, 'answer': answer, 'topic': topic})
                    except: 
                        try: #pick up questions that did not indent the code block
                            solution_new = ""
                            lines = solution.split('\n')
                            indent = False
                            for line in lines:
                                if line.startswith('def solution():'):
                                    solution_new += line + '\n'

                                if not line.startswith('def solution():') and not line.startswith('    return result'):
                                    solution_new += '    ' + line + '\n'

                                if line.startswith('    return result'):
                                    solution_new += line
                            solution_text = solution_new
                            # Execute the Python code
                            exec(solution_new)
                            answer = solution()
                            if answer==None:
                                continue
                            data.append({"question": question, "solution": solution_text, 'answer': answer, "topic": topic})
                        except:
                                pass
                except:
                    pass
            if "return" in solution:
                try: 
                    solution_text = solution
                    try:
                        # Execute the Python code
                        exec(solution)
                        answer = solution()
                        if answer==None:
                            continue
                        data.append({"question": question, "solution": solution_text, 'answer': answer, "topic": topic})
                    except: 
                        try: #pick up questions that did not indent the code block
                            solution_new = ""
                            lines = solution.split('\n')
                            indent = False
                            for line in lines:
                                if line.startswith('def solution():'):
                                    solution_new += line + '\n'

                                if not line.startswith('def solution():') and not line.startswith('    return result'):
                                    solution_new += '    ' + line + '\n'

                                if line.startswith('    return result'):
                                    solution_new += line
                            solution_text = solution_new
                            # Execute the Python code
                            exec(solution_new)
                            answer = solution()
                            if answer==None:
                                continue
                            data.append({"question": question, "solution": solution_text, 'answer': answer, "topic": topic})
                        except:
                            pass
                except:
                    pass
        except:
            pass
df3 = pd.DataFrame.from_dict(data)
result1 = "Percentage of Questions with Executable Code: " + str((len(df3)/total_questions)*100)
result2 = "Percentage of Questions with Executable Code (excluding non generations): " + str((len(df3)/non_zero)*100)
print(result1)
print(result2)
print(len(df3), total_questions)

Percentage of Questions with Executable Code: 20.45
Percentage of Questions with Executable Code (excluding non generations): 87.20682302771856
409 2000


In [74]:
print(f"Percentage of questions that have 0 as answer: {(len(df3[df3['answer']==0])/len(df3))*100}")

Percentage of questions that have 0 as answer: 5.623471882640587


In [75]:
df3 = df3.drop_duplicates(subset='question')
df3 = df3[~df3['question'].isin(finetune['question'])]
question_numbers = []
for i in range(0, len(df3)):
    question_numbers.append(f"mathwell8b_two_stage_q{i}")
df3['question_number'] = question_numbers
df3_an = df3.sample(n=min(250,len(df3)), random_state = 42)

In [76]:
df3

,question,solution,answer,topic,question_number
0,A Unicorn breeder is collecting unicorns. He h...,def solution():\n #The Unicorn breeder star...,250,unicorns,mathwell8b_two_stage_q0
1,A scoop of vanilla ice cream is $3 and a scoop...,def solution():\n #A scoop of vanilla ice c...,3,ice cream,mathwell8b_two_stage_q1
2,Serena Williams is a 4-time Olympic gold medal...,def solution():\n #Serena has 3 gold medals...,0,Serena Williams,mathwell8b_two_stage_q2
3,LeBron James has 50000 points. He averages 30 ...,def solution():\n #LeBron James has 50000 p...,666.666667,LeBron James,mathwell8b_two_stage_q3
4,Barbie has 2000 dolls. She sells 300 of them i...,def solution():\n #Barbie started with 2000...,1000,Barbie,mathwell8b_two_stage_q4
...,...,...,...,...,...
404,Superman can fly 1000 miles per hour. He is 3...,def solution():\n #Superman can fly 1000 mi...,3.0,Superman,mathwell8b_two_stage_q389
405,Superman is 1000 times stronger than a human. ...,def solution():\n #A human can lift 100 pou...,100000,Superman,mathwell8b_two_stage_q390
406,1000 cars are in a parking lot. 200 of the car...,def solution():\n #1000 cars are in a parki...,400,cars,mathwell8b_two_stage_q391
407,1000000000 V-Bucks is the in-game currency in ...,def solution():\n #1000000000 V-Bucks is 10...,900000000,Fortnite,mathwell8b_two_stage_q392


In [55]:
#df3 = df3[~df3['question'].isin(df3_an['question'])]
# extra_an2 = df2.sample(n=250-len(df2_an), random_state = 42)
# extra_an2

## MATHWELL 8B KTO Few Shot

In [2]:
file_path = 'mathwell8b_kto_questions_few.txt'
data = []

with open(file_path, 'r') as file:
    text = file.read()
    sections = text.split("Topic: ")
    #sections = text.split("### Response:")
    del sections[0]  # Remove the initial part before the question
    total_questions = len(sections)
    non_zero = total_questions
    for section in sections:
        if "Question" not in section:
            non_zero-=1
        try:
            topic = section.split("### Response:")[0].strip()
            section = section.split("### Response:")[1].strip()
            question = section.split("Question:")[1].strip()
            question = question.split("Solution:")[0].strip()
            solution = section.split("Solution:")[1].strip()
            if 'while' in solution:
                pass
            if "##" in question:
                question = question.split("##")[0]
#             if "\n\n" in question:
#                 question = question.split("\n\n")
#                 question = "\n".join(question)
            if "\nBel" in solution:
                solution = solution.split("\nBel")[0]
            if "##" in solution:
                solution = solution.split("##")[0]
            if "return" not in solution:
                try: 
                    solution = solution + '\n' + '    return result'
                    solution_text = solution
                    try:
                            # Execute the Python code
                        exec(solution)
                        answer = solution()
                        if answer==None:
                            continue
                        data.append({"question": question, "solution": solution_text, 'answer': answer, 'topic': topic})
                    except: 
                        try: #pick up questions that did not indent the code block
                            solution_new = ""
                            lines = solution.split('\n')
                            indent = False
                            for line in lines:
                                if line.startswith('def solution():'):
                                    solution_new += line + '\n'

                                if not line.startswith('def solution():') and not line.startswith('    return result'):
                                    solution_new += '    ' + line + '\n'

                                if line.startswith('    return result'):
                                    solution_new += line
                            solution_text = solution_new
                            # Execute the Python code
                            exec(solution_new)
                            answer = solution()
                            if answer==None:
                                continue
                            data.append({"question": question, "solution": solution_text, 'answer': answer, "topic": topic})
                        except:
                                pass
                except:
                    pass
            if "return" in solution:
                try: 
                    solution_text = solution
                    try:
                        # Execute the Python code
                        exec(solution)
                        answer = solution()
                        if answer==None:
                            continue
                        data.append({"question": question, "solution": solution_text, 'answer': answer, "topic": topic})
                    except: 
                        try: #pick up questions that did not indent the code block
                            solution_new = ""
                            lines = solution.split('\n')
                            indent = False
                            for line in lines:
                                if line.startswith('def solution():'):
                                    solution_new += line + '\n'

                                if not line.startswith('def solution():') and not line.startswith('    return result'):
                                    solution_new += '    ' + line + '\n'

                                if line.startswith('    return result'):
                                    solution_new += line
                            solution_text = solution_new
                            # Execute the Python code
                            exec(solution_new)
                            answer = solution()
                            if answer==None:
                                continue
                            data.append({"question": question, "solution": solution_text, 'answer': answer, "topic": topic})
                        except:
                            pass
                except:
                    pass
        except:
            pass
df4 = pd.DataFrame.from_dict(data)
result1 = "Percentage of Questions with Executable Code: " + str((len(df4)/total_questions)*100)
result2 = "Percentage of Questions with Executable Code (excluding non generations): " + str((len(df4)/non_zero)*100)
print(result1)
print(result2)
print(len(df4), total_questions)

Percentage of Questions with Executable Code: 79.88898377455168
Percentage of Questions with Executable Code (excluding non generations): 92.34945705824285
1871 2342


In [3]:
print(f"Percentage of questions that have 0 as answer: {(len(df4[df4['answer']==0])/len(df4))*100}")

Percentage of questions that have 0 as answer: 3.7947621592731156


In [4]:
df4 = df4.drop_duplicates(subset='question')
print(len(df4))
df4 = df4[~df4['question'].isin(finetune['question'])]
print(len(df4))
question_numbers = []
for i in range(0, len(df4)):
    question_numbers.append(f"mathwell8b_kto_q{i}")
df4['question_number'] = question_numbers
df4_an = df4.sample(n=min(250,len(df4)), random_state = 42)
df4

1772
1655


,question,solution,answer,topic,question_number
0,Taylor Swift has 2000 fans. 1000 of them are i...,def solution():\n #Total fans\n total_fa...,500,Taylor Swift,mathwell8b_kto_q0
1,Super Mario has 300 coins. He spends 50 coins ...,def solution():\n #Super Mario started with...,200,Super Mario,mathwell8b_kto_q1
3,Thor has 2000 lightning bolts. He uses 1000 of...,def solution():\n #Thor started with 2000 l...,300,"Thor, the God of Thunder",mathwell8b_kto_q2
4,Patrick Mahomes is the quarterback for the Kan...,def solution():\n #Patrick Mahomes has 3000...,2800,Patrick Mahomes,mathwell8b_kto_q3
5,Steph Curry is a basketball player. He made 12...,def solution():\n #Steph Curry made 12 thre...,120.0,Steph Curry,mathwell8b_kto_q4
...,...,...,...,...,...
1866,Steph Curry has 2000 3-pointers. He has 1000 m...,def solution():\n #Steph Curry has 2000 3-p...,17000,basketball,mathwell8b_kto_q1650
1867,Taylor Swift has 2000 fans. 1000 of them are i...,def solution():\n #Total fans\n fans_tot...,500,Taylor Swift,mathwell8b_kto_q1651
1868,Serena Williams has 18 Grand Slam titles. 13 o...,def solution():\n #Serena Williams has 18 G...,4,Serena Williams,mathwell8b_kto_q1652
1869,1200 cookies are in a jar. 200 of the cookies ...,def solution():\n #Total number of cookies\...,0,cookies,mathwell8b_kto_q1653


# Explore Random Samples

In [6]:
import random
for i in range(0, 10):
    index = random.randint(0, len(df4))
    question = df4.iloc[index]['question']
    solution = df4.iloc[index]['solution']
    print(f"Question: {question}\n\nSolution:\n{solution}")

Question: Iron Man has 2000 gold coins. He spends 500 gold coins on a new suit of armor. He then spends 200 gold coins on a new AI. He spends 300 gold coins on a new repulsor beam. How many gold coins does he have left?

Solution:
def solution():
    #Iron Man started with 2000 gold coins
    gold_coins_initial = 2000
    #He spent 500 on a new suit of armor
    gold_coins_suit = 500
    #He spent 200 on a new AI
    gold_coins_ai = 200
    #He spent 300 on a new repulsor beam
    gold_coins_repulsor = 300
    #The answer is
    result = gold_coins_initial - gold_coins_suit - gold_coins_ai - gold_coins_repulsor
    return result
Question: Steph Curry, the point guard for the Golden State Warriors, is the best 3-point shooter in the NBA. He has made 3-pointers in 24 consecutive games. In each of the first 12 games, he made 5 3-pointers. In each of the next 12 games, he made 6 3-pointers. How many 3-pointers did he make in the 24 games?

Solution:
def solution():
    #Steph Curry made 5 

In [59]:
import random
for i in range(0, 10):
    index = random.randint(0, len(df2))
    question = df2.iloc[index]['question']
    solution = df2.iloc[index]['solution']
    print(f"Question: {question}\n\nSolution:\n{solution}")

Question: A football team has 100 players. 

40 of the players are on the offense. 

35 of the players are on the defense. 

The rest of the players are on special teams. 

How many players are on special teams?

Solution:
def solution():
    #A football team has 100 players
    players_total = 100
    #40 of the players are on the offense
    players_offense = 40
    #35 of the players are on the defense
    players_defense = 35
    #The answer is
    players_special_teams = players_total - players_offense - players_defense
    return players_special_teams
Question: Serena Williams has 200 Wimbledon titles. 100 of them are in singles, 50 are in doubles, and the rest are in mixed doubles. How many Wimbledon titles does she have in mixed doubles?

Solution:
def solution():
    #Serena Williams has 200 Wimbledon titles
    total_titles = 200
    #100 of them are in singles
    singles_titles = 100
    #50 are in doubles
    doubles_titles = 50
    #The rest are in mixed doubles
    mixed

In [60]:
import random
for i in range(0, 10):
    index = random.randint(0, len(df3))
    question = df3.iloc[index]['question']
    solution = df3.iloc[index]['solution']
    print(f"Question: {question}\n\nSolution:\n{solution}")

Question: Batman is 55 years old. He was 25 years old when he became Batman. How long has he been Batman?

Solution:
def solution():
    #Batman is 55 years old
    batman_age = 55
    #He was 25 years old when he became Batman
    batman_age_became = 25
    #The answer is
    result = batman_age - batman_age_became
    return result
Question: The Lacrosse team has 15 players.  They need 3 sticks each.  How many sticks do they need in total?

Solution:
def solution():
    #The Lacrosse team has 15 players
    players = 15
    #They need 3 sticks each
    sticks_per_player = 3
    #The answer is
    result = players * sticks_per_player
    return result
Question: Steph Curry is a basketball player. He made 50 three-pointers in the first game. He made 100 three-pointers in the second game. He made 200 three-pointers in the third game. How many three-pointers did he make in total?

Solution:
def solution():
    #Steph Curry made 50 three-pointers in the first game
    three_pointers_first

In [63]:
17/20

0.85